# Estudo sobre a base do jogo Jokenpower

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('attacks_pattern.csv')
df

,timestamp,time_since_start,damaged_player,attack_type,damage,is_counter_attack
0,2023-03-18 22:18:58.246426,3453,Main,Rock,10,False
1,2023-03-18 22:18:58.361588,3569,Enemy,Rock,10,False
2,2023-03-18 22:18:59.762810,4970,Main,Paper,1,False
3,2023-03-18 22:19:00.747199,5954,Enemy,Scissors,3,False
4,2023-03-18 22:19:01.854446,7061,Main,Rock,10,False
5,2023-03-18 22:19:02.575044,7782,Enemy,Paper,1,False
6,2023-03-18 22:19:03.359242,8566,Main,Paper,1,False
7,2023-03-18 22:19:05.045920,10253,Enemy,Rock,20,True
8,2023-03-18 22:19:06.062385,11269,Enemy,Rock,10,False
9,2023-03-18 22:19:06.833887,12040,Enemy,Rock,20,True


## Tratamento de dados

In [3]:
# Parsear string timestamp para Tipo Timestamp do pandas
df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [4]:
# Criar campo com diferença de tempo entre ataques
df['time_diff'] = df['timestamp'].diff().dt.total_seconds()

In [5]:
# Definir início de partida: quando próximo time_since_start for menor que anterior
df.loc[df['time_since_start'].diff() < 0, 'time_diff'] = np.nan

In [6]:
# Deletar NaN da tabela
df.dropna(inplace=True)

In [7]:
# Normalizando diferença de tempo entre ataques
df['label_time_diff'] = df['time_diff'].apply(lambda x: int(x*10))

In [8]:
# Normalizando valores qualitativos para quantitativos

def get_norm_attack_type(atk: str):
    result = 99
    if atk == 'Rock':
        result = 0
    elif atk == 'Paper':
        result = 1
    elif atk == 'Scissors':
        result = 2
    return result

df['label_attack_type'] = df['attack_type'].apply(get_norm_attack_type)

In [9]:
df

,timestamp,time_since_start,damaged_player,attack_type,damage,is_counter_attack,time_diff,label_time_diff,label_attack_type
1,2023-03-18 22:18:58.361588,3569,Enemy,Rock,10,False,0.115162,1,0
2,2023-03-18 22:18:59.762810,4970,Main,Paper,1,False,1.401222,14,1
3,2023-03-18 22:19:00.747199,5954,Enemy,Scissors,3,False,0.984389,9,2
4,2023-03-18 22:19:01.854446,7061,Main,Rock,10,False,1.107247,11,0
5,2023-03-18 22:19:02.575044,7782,Enemy,Paper,1,False,0.720598,7,1
6,2023-03-18 22:19:03.359242,8566,Main,Paper,1,False,0.784198,7,1
7,2023-03-18 22:19:05.045920,10253,Enemy,Rock,20,True,1.686678,16,0
8,2023-03-18 22:19:06.062385,11269,Enemy,Rock,10,False,1.016465,10,0
9,2023-03-18 22:19:06.833887,12040,Enemy,Rock,20,True,0.771502,7,0
10,2023-03-18 22:19:07.862736,13062,Enemy,Rock,10,False,1.028849,10,0


## Scikit Learn

In [10]:
# Pegar apenas danos ao inimigo = Apenas do player
df_only_player = df.loc[df['damaged_player'] == 'Enemy']

In [11]:
# Filtrar apenas colunas necessárias para fator X
x_chance_contra_ataque = df_only_player.loc[:, ['label_attack_type', 'label_time_diff']].values
x_chance_contra_ataque

array([[ 0,  1],
       [ 2,  9],
       [ 1,  7],
       [ 0, 16],
       [ 0, 10],
       [ 0,  7],
       [ 0, 10],
       [ 0,  7],
       [ 0, 10],
       [ 2, 12],
       [ 1,  7],
       [ 0,  9],
       [ 0, 10],
       [ 1,  3],
       [ 0,  7],
       [ 1, 11],
       [ 2, 10],
       [ 2,  3],
       [ 2,  3],
       [ 1,  4],
       [ 0, 15],
       [ 2,  4]])

In [12]:
# Filtrar apenas coluna necessária para fator Y
y_chance_contra_ataque = df_only_player.loc[:, 'is_counter_attack'].values
y_chance_contra_ataque

array([False, False, False,  True, False,  True, False,  True, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False])

### Normalização de valores qualitativos para quantitativos

In [13]:
# Normalização deve ser feita para o modelo GaussianNB aceitar como input

# from sklearn.preprocessing import LabelEncoder
# label_encoder_attack_type = LabelEncoder()
# x_chance_contra_ataque[:,0] = label_encoder_attack_type.fit_transform(x_chance_contra_ataque[:,0])

### Treinamento do modelo com algoritmo Naive Bayes

In [14]:
from sklearn.naive_bayes import GaussianNB

In [15]:
naive_chance_contra_ataque = GaussianNB()
naive_chance_contra_ataque.fit(x_chance_contra_ataque, y_chance_contra_ataque)

GaussianNB()

### Previsão

In [16]:
# utilizando o modelo: de acordo com o input de teste, prever se irá contra-atacar
# [[0, 7]] -> [['Rock', time_diff]]
previsao = naive_chance_contra_ataque.predict([[0, 7]])
previsao

array([ True])

# Gráficos de Comparação

In [18]:
# Pegar apenas danos ao inimigo = Apenas do player
df_only_player 

,timestamp,time_since_start,damaged_player,attack_type,damage,is_counter_attack,time_diff,label_time_diff,label_attack_type
1,2023-03-18 22:18:58.361588,3569,Enemy,Rock,10,False,0.115162,1,0
3,2023-03-18 22:19:00.747199,5954,Enemy,Scissors,3,False,0.984389,9,2
5,2023-03-18 22:19:02.575044,7782,Enemy,Paper,1,False,0.720598,7,1
7,2023-03-18 22:19:05.045920,10253,Enemy,Rock,20,True,1.686678,16,0
8,2023-03-18 22:19:06.062385,11269,Enemy,Rock,10,False,1.016465,10,0
9,2023-03-18 22:19:06.833887,12040,Enemy,Rock,20,True,0.771502,7,0
10,2023-03-18 22:19:07.862736,13062,Enemy,Rock,10,False,1.028849,10,0
11,2023-03-18 22:19:08.644172,13852,Enemy,Rock,20,True,0.781436,7,0
12,2023-03-18 22:19:09.674454,14881,Enemy,Rock,10,False,1.030282,10,0
14,2023-03-18 22:19:11.835821,17043,Enemy,Scissors,3,False,1.260471,12,2
